In [1]:

from google.colab import drive
import os #Acessa e gerencia as pastas no drive
import cv2 # Processamento de imagens
import numpy as np #Transforma arrays em dados numericos
from tensorflow.keras.preprocessing.image import img_to_array #Converte imagens em arrays.
from tensorflow.keras.applications import VGG16 #Importa modelo de rede neural.
from tensorflow.keras.models import Model #Importa camada da redeneural em modelo estatistico.
from tensorflow.keras.layers import Dense, Flatten #Importa camada naa rede neural para transformar vetores em numeros unicos e realizar classificação final.
from sklearn.preprocessing import LabelEncoder #Converte rótulos (nomes das classes) em valores numéricos.
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Função para carregar e redimensionar as imagens
def load_images_from_folder(folder, image_size=(224, 224)):
    images = []
    labels = []
    for filename in os.listdir(folder):  # Percorre os arquivos diretamente
        img_path = os.path.join(folder, filename)
        if filename.endswith(('.jpeg', '.jpg', '.png')):  # Aceita mais formatos
            img = cv2.imread(img_path)
            if img is not None:  # Garante que a imagem foi carregada corretamente
                img = cv2.resize(img, image_size)  # Redimensiona a imagem
                img = img_to_array(img)  # Converte para array
                images.append(img)
                labels.append(folder.split("/")[-1])  # Usa o nome da pasta como label
    return np.array(images), np.array(labels)

def predict_image(image_path,model):
    # Carregar e preparar a imagem
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))  # Redimensionar para o formato da VGG16
    img = img_to_array(img) / 255.0  # Normalizar
    img = np.expand_dims(img, axis=0)  # Adicionar dimensão extra para o modelo

    prediction = model.predict(img)
    label = "Basquete" if prediction[0][0] > 0.5 else "Futebol"
    print(f"A imagem foi classificada como {label}")


# Montar o Google Drive
drive.mount('/content/drive')

# Caminho para a pasta do dataset
dataset_path = "/content/drive/MyDrive/Projeto Transfer Learning/dataset_esportes/"

# Listar arquivos dentro de cada categoria
basquete_path = os.path.join(dataset_path, "basquete")
futebol_path = os.path.join(dataset_path, "futebol")

print("Imagens de basquete:", os.listdir(basquete_path))
print("Imagens de futebol:", os.listdir(futebol_path))


# Defina o caminho correto da pasta
dataset_path = "/content/drive/MyDrive/Projeto Transfer Learning/dataset_esportes"

# Carregar as imagens de basquete e futebol
basquete_images, basquete_labels = load_images_from_folder(basquete_path)
futebol_images, futebol_labels = load_images_from_folder(futebol_path)

images = np.concatenate((basquete_images, futebol_images), axis=0)
labels = np.concatenate((basquete_labels, futebol_labels), axis=0)

# Normalizar as imagens
images = images.astype("float32") / 255.0

# Codificar os rótulos
le = LabelEncoder()
labels = le.fit_transform(labels)

print("Imagens carregadas e redimensionadas com sucesso!")
print("Total de imagens:", len(images))

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split (images, labels, test_size=0.2, random_state=42)
print("Total de imagens de treino:", len(X_train))
print("Total de imagens de teste:", len(X_test))


# Carregar o modelo pré-treinado VGG16 sem as camadas superiores (include_top=False)
base_model = VGG16(weights="imagenet", include_top= False, input_shape=(224, 224, 3))
base_model.trainable = False

x = Flatten()(base_model.output)  # Achata a saída da VGG16
x = Dense(256, activation="relu")(x)  # Camada densa intermediária
x = Dense(1, activation="sigmoid")(x)  # Camada final binária (basquete ou futebol)

model = Model(inputs=base_model.input, outputs=x)

model.summary()

model.compile(
    loss= "binary_crossentropy",
    optimizer= Adam(learning_rate=0.0001),
    metrics = ["accuracy"]
    )
history= model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=8
)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Acuracia no conjunto de teste:{accuracy:.2f}%")

plt.figure(figsize=(12, 6))

# Gráfico de Acurácia
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Acurácia de Treinamento')
plt.plot(history.history['val_accuracy'], label='Acurácia de Validação')
plt.title('Evolução da Acurácia')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()

# Gráfico de Perda
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Perda de Treinamento')
plt.plot(history.history['val_loss'], label='Perda de Validação')
plt.title('Evolução da Perda')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend()

plt.show()

predict_image("/content/drive/MyDrive/Projeto Transfer Learning/dataset_esportes/testes/teste1.jpg", model)

model.save("/content/drive/MyDrive/Projeto Transfer Learning/transfer_learning_model.keras")

drive  sample_data
Refresh index: 100% (67/67), done.
^C
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Imagens de basquete: ['r1160080_1296x729_16-9.jpg', 'nba_rank_top10_cr_16x9_800x450.jpg', '063_1245255600.jpg', 'Lebron-James-Getty.jpg', 'money-basquete-ja-morant.jpg', 'victor-wembanyama-moritz-wagner-100924.jpg', '42e652c18bf25d1531625d2d96191c88.jpg', 'usatsi-23174159.jpg', 'merlin_215266479_ef43c8dc-6eab-4f28-a4b9-c3928e56536b-superJumbo.jpg', 'Kyrie-Irving_Dallas-Mavericks.jpg', 'irving.jpg', 'trae-young-4-cancelled-adidas-contract-officially-terminated-v0-bxrt1gokoj0e1.jpg', 'images.jpg', '1232450492.jpg', 'USATSI_25322143_168414878_lowres-scaled.jpg', 'Russell_Westbrook_(March_21,_2022)_(cropped).jpg', 'images (1).jpg', '00nba-westbrook-opener-mediumSquareAt3X.jpg', 'images (2).jpg', 'doncic-durant-768x432.jpg', 'suns_vs_blazers-2_3_25-kd.jpg', 'Curry_s_bio_997380.jpg']
Imagens de futebol: ['ima

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 21,137,729 (80.63 MB)

 Trainable params: 6,423,041 (24.50 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.3513 - loss: 1.0507

KeyboardInterrupt: 